In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import timedelta
from utils.dataloader import Dataloader

# Initialize Dataloader and load all datasets
dataloader = Dataloader(data_dir="data/")
data = dataloader.load_all()

df_covid_comprehensive = data["comprehensive_data"]
print("Columns in df_covid_comprehensive:", df_covid_comprehensive.columns)
# df_covid_comprehensive['Date'] = pd.to_datetime(df_covid_comprehensive['Date']) # Temporarily commented out for column inspection

df_testing = data["covid19_testing_record"]
df_testing['date'] = pd.to_datetime(df_testing['date'])

df_variants = data["covid19_variants_found"]
df_variants['date'] = pd.to_datetime(df_variants['date'])

df_vacc_country = data["vaccinations_by_country"]
df_vacc_country['date'] = pd.to_datetime(df_vacc_country['date'])

df_vacc_manufacturer = data["vaccination_by_manufacturer"]
df_vacc_manufacturer['date'] = pd.to_datetime(df_vacc_manufacturer['date'])

print("All data loaded successfully.")

Columns in df_covid_comprehensive: Index(['ID', 'country', 'confirmed_cases', 'deaths_cases', 'recovered_cases',
       'is_western_europe'],
      dtype='object')
All data loaded successfully.


# Vaccine Performance Calculation by Manufacturer - Implementation

## Approach 1: Comparative Effectiveness Analysis (Market Share vs. Health Outcomes)

In [2]:
### Phase 1: Data Preparation & Population Estimation

# Placeholder for population data. In a real scenario, this would come from a reliable external source.
COUNTRY_POPULATIONS = {
    'United Kingdom': 67000000,
    'France': 65000000,
    'Germany': 83000000,
    'Italy': 59000000,
    'Spain': 47000000,
    'Netherlands': 17000000,
    'Belgium': 11500000,
    'Ireland': 5000000,
    'Luxembourg': 630000,
    'Monaco': 39000,
    'United States': 330000000,
    'Canada': 38000000,
    'Australia': 25000000,
    'India': 1380000000,
    'Brazil': 212000000,
    'South Africa': 59000000,
    'Japan': 126000000,
    'South Korea': 51000000,
    'China': 1440000000,
    'Russia': 145000000
}

def get_population(country):
    """
    Retrieves the population for a given country.
    Assumes a fixed population for demonstration purposes.
    """
    return COUNTRY_POPULATIONS.get(country, 10000000) # Default to 10M if not found

def calculate_population_segments(country, date):
    """
    Estimates vaccinated vs unvaccinated populations by manufacturer for a given country and date.
    Assumes 'total_vaccinations' from manufacturer data represents unique individuals for simplicity.
    """
    total_pop = get_population(country)
    
    # Get latest vaccination data by manufacturer up to the specified date
    vacc_by_mfr_filtered = df_vacc_manufacturer[
        (df_vacc_manufacturer['country'] == country) &
        (df_vacc_manufacturer['date'] <= date)
    ].copy()
    
    # Get the latest total_vaccinations for each manufacturer
    latest_vacc_by_mfr = vacc_by_mfr_filtered.groupby('manufacturer')['total_vaccinations'].max().fillna(0)
    
    # Get total people fully vaccinated from country-level data for a more accurate 'vaccinated' count
    # This helps account for second doses and avoids double-counting if a person received vaccines from different manufacturers
    # However, for manufacturer-specific VE, we still need to attribute based on manufacturer data.
    # For simplicity, we'll use the sum of manufacturer vaccinations as a proxy for total vaccinated by manufacturer.
    # A more rigorous approach would require individual-level data or complex modeling.
    
    # Sum of vaccinations by manufacturer (can exceed total_pop if multiple doses are counted as 'vaccinations')
    # We'll cap this at total_pop for 'vaccinated' population estimation.
    total_vaccinated_by_manufacturer_sum = latest_vacc_by_mfr.sum()
    
    # Use people_fully_vaccinated from df_vacc_country as the best estimate for total vaccinated individuals
    # This is a crucial assumption for the 'unvaccinated' denominator.
    country_vacc_data = df_vacc_country[
        (df_vacc_country['country'] == country) &
        (df_vacc_country['date'] <= date)
    ].sort_values('date', ascending=False).head(1)
    
    total_people_fully_vaccinated = country_vacc_data['people_fully_vaccinated'].iloc[0] if not country_vacc_data.empty else 0
    
    # Ensure total_people_fully_vaccinated does not exceed total_pop
    total_people_fully_vaccinated = min(total_people_fully_vaccinated, total_pop)

    unvaccinated_pop = max(0, total_pop - total_people_fully_vaccinated)
    
    # Distribute the 'total_people_fully_vaccinated' proportionally among manufacturers
    # This is an assumption: we don't have individual-level data to know exact manufacturer distribution
    manufacturer_proportions = latest_vacc_by_mfr / total_vaccinated_by_manufacturer_sum if total_vaccinated_by_manufacturer_sum > 0 else 0
    
    vaccinated_by_manufacturer = (manufacturer_proportions * total_people_fully_vaccinated).to_dict()
    
    return vaccinated_by_manufacturer, unvaccinated_pop, total_pop

### Phase 2: Outcome Attribution

def get_dominant_vaccine(country, date_threshold=None):
    """
    Determines the dominant vaccine manufacturer for a given country.
    Dominance is based on the highest total vaccinations by manufacturer.
    If date_threshold is provided, it considers data up to that date.
    """
    df_filtered = df_vacc_manufacturer[df_vacc_manufacturer['country'] == country].copy()
    if date_threshold:
        df_filtered = df_filtered[df_filtered['date'] <= date_threshold]
    
    if df_filtered.empty:
        return None
    
    # Get the latest total vaccinations for each manufacturer
    latest_vacc_by_mfr = df_filtered.groupby('manufacturer')['total_vaccinations'].max().fillna(0)
    
    if latest_vacc_by_mfr.empty or latest_vacc_by_mfr.sum() == 0:
        return None
        
    dominant_vaccine = latest_vacc_by_mfr.idxmax()
    return dominant_vaccine

def calculate_infection_rates_by_manufacturer(countries_to_analyze, start_date_map):
    """
    Calculates manufacturer-specific infection rates for countries with dominant manufacturers.
    Assumes proportional case distribution based on vaccination status.
    """
    results = {}
    
    for country in countries_to_analyze:
        vaccination_start_date = start_date_map.get(country)
        if not vaccination_start_date:
            print(f"Skipping {country}: Vaccination start date not found.")
            continue
            
        dominant_vaccine = get_dominant_vaccine(country, date_threshold=vaccination_start_date + timedelta(days=90)) # Consider dominance after 3 months of rollout
        if not dominant_vaccine:
            print(f"Skipping {country}: No dominant vaccine found.")
            continue
            
        # Get infection data after vaccination rollout
        post_vacc_period = df_covid_comprehensive[
            (df_covid_comprehensive['country'] == country) &
            (df_covid_comprehensive['Date'] >= vaccination_start_date) # Corrected from 'date' to 'Date'
        ].copy()
        
        if post_vacc_period.empty:
            print(f"Skipping {country}: No comprehensive data post vaccination start.")
            continue
            
        # Calculate daily new cases
        post_vacc_period['daily_cases'] = post_vacc_period['confirmed_cases'].diff().fillna(0).clip(lower=0)
        
        # Get population segments for the latest date in the post-vaccination period
        latest_date_for_pop = post_vacc_period['Date'].max() # Corrected from 'date' to 'Date'
        vaccinated_by_manufacturer_dict, unvaccinated_pop, total_pop = calculate_population_segments(country, latest_date_for_pop)
        
        vaccinated_pop_dominant = vaccinated_by_manufacturer_dict.get(dominant_vaccine, 0)
        
        # Total cases during the post-vaccination period
        total_cases_post_vacc = post_vacc_period['daily_cases'].sum()
        
        # Assumption: Proportional case distribution based on estimated population segments
        # This is a strong assumption and a limitation without individual-level data.
        if (vaccinated_pop_dominant + unvaccinated_pop) == 0:
            print(f"Skipping {country}: Total population for infection rate calculation is zero.")
            continue
            
        # Estimate infections in vaccinated and unvaccinated groups
        estimated_infections_vaccinated = total_cases_post_vacc * (vaccinated_pop_dominant / (vaccinated_pop_dominant + unvaccinated_pop)) if (vaccinated_pop_dominant + unvaccinated_pop) > 0 else 0
        estimated_infections_unvaccinated = total_cases_post_vacc * (unvaccinated_pop / (vaccinated_pop_dominant + unvaccinated_pop)) if (vaccinated_pop_dominant + unvaccinated_pop) > 0 else 0
        
        # Calculate rates per 100,000
        infection_rate_vaccinated = (estimated_infections_vaccinated / vaccinated_pop_dominant) * 100000 if vaccinated_pop_dominant > 0 else 0
        infection_rate_unvaccinated = (estimated_infections_unvaccinated / unvaccinated_pop) * 100000 if unvaccinated_pop > 0 else 0
        
        results[dominant_vaccine] = {
            'country': country,
            'infection_rate_vaccinated': infection_rate_vaccinated,
            'infection_rate_unvaccinated': infection_rate_unvaccinated,
            'vaccinated_pop': vaccinated_pop_dominant,
            'unvaccinated_pop': unvaccinated_pop,
            'total_cases': total_cases_post_vacc
        }
        
    return results

def calculate_death_rates_by_manufacturer(countries_to_analyze, start_date_map):
    """
    Calculates manufacturer-specific death rates for countries with dominant manufacturers.
    Assumes proportional death distribution based on vaccination status.
    """
    results = {}
    
    for country in countries_to_analyze:
        vaccination_start_date = start_date_map.get(country)
        if not vaccination_start_date:
            continue
            
        dominant_vaccine = get_dominant_vaccine(country, date_threshold=vaccination_start_date + timedelta(days=90))
        if not dominant_vaccine:
            continue
            
        post_vacc_period = df_covid_comprehensive[
            (df_covid_comprehensive['country'] == country) &
            (df_covid_comprehensive['Date'] >= vaccination_start_date) # Corrected from 'date' to 'Date'
        ].copy()
        
        if post_vacc_period.empty:
            continue
            
        post_vacc_period['daily_deaths'] = post_vacc_period['deaths_cases'].diff().fillna(0).clip(lower=0)
        
        latest_date_for_pop = post_vacc_period['Date'].max() # Corrected from 'date' to 'Date'
        vaccinated_by_manufacturer_dict, unvaccinated_pop, total_pop = calculate_population_segments(country, latest_date_for_pop)
        
        vaccinated_pop_dominant = vaccinated_by_manufacturer_dict.get(dominant_vaccine, 0)
        
        total_deaths_post_vacc = post_vacc_period['daily_deaths'].sum()
        
        if (vaccinated_pop_dominant + unvaccinated_pop) == 0:
            continue
            
        estimated_deaths_vaccinated = total_deaths_post_vacc * (vaccinated_pop_dominant / (vaccinated_pop_dominant + unvaccinated_pop)) if (vaccinated_pop_dominant + unvaccinated_pop) > 0 else 0
        estimated_deaths_unvaccinated = total_deaths_post_vacc * (unvaccinated_pop / (vaccinated_pop_dominant + unvaccinated_pop)) if (vaccinated_pop_dominant + unvaccinated_pop) > 0 else 0
        
        death_rate_vacc = (estimated_deaths_vaccinated / vaccinated_pop_dominant) * 100000 if vaccinated_pop_dominant > 0 else 0
        death_rate_unvacc = (estimated_deaths_unvaccinated / unvaccinated_pop) * 100000 if unvaccinated_pop > 0 else 0
        
        results[dominant_vaccine] = {
            'country': country,
            'death_rate_vaccinated': death_rate_vacc,
            'death_rate_unvaccinated': death_rate_unvacc,
            'vaccinated_pop': vaccinated_pop_dominant,
            'unvaccinated_pop': unvaccinated_pop,
            'total_deaths': total_deaths_post_vacc
        }
        
    return results

### Phase 3: Vaccine Effectiveness Calculation

def calculate_ci(risk_ratio, n_exposed, n_unexposed, cases_exposed, cases_unexposed, alpha=0.05):
    """
    Calculates the Wald confidence interval for a risk ratio.
    Requires counts of cases and populations for both groups.
    """
    if cases_exposed == 0 or cases_unexposed == 0 or n_exposed == 0 or n_unexposed == 0:
        return (np.nan, np.nan) # Cannot calculate CI if any count is zero

    # Calculate standard error of the log risk ratio
    se_log_rr = np.sqrt(1/cases_exposed - 1/n_exposed + 1/cases_unexposed - 1/n_unexposed)
    
    # Z-score for desired confidence level
    z = norm.ppf(1 - alpha/2)
    
    # Confidence interval for log risk ratio
    log_rr = np.log(risk_ratio)
    lower_log_rr = log_rr - z * se_log_rr
    upper_log_rr = log_rr + z * se_log_rr
    
    # Convert back to risk ratio scale
    lower_rr = np.exp(lower_log_rr)
    upper_rr = np.exp(upper_log_rr)
    
    # Convert to VE scale
    lower_ve = (1 - upper_rr) * 100
    upper_ve = (1 - lower_rr) * 100
    
    return (lower_ve, upper_ve)

def calculate_vaccine_effectiveness(infection_data, death_data):
    """
    Calculates vaccine effectiveness (VE) against infection and death.
    """
    ve_results = {}
    
    all_manufacturers = set(infection_data.keys()).union(set(death_data.keys()))
    
    for vaccine in all_manufacturers:
        ve_infection = np.nan
        ve_death = np.nan
        ci_infection = (np.nan, np.nan)
        ci_death = (np.nan, np.nan)
        
        # VE against infection
        if vaccine in infection_data:
            data_inf = infection_data[vaccine]
            inf_rate_vacc = data_inf['infection_rate_vaccinated']
            inf_rate_unvacc = data_inf['infection_rate_unvaccinated']
            
            if inf_rate_unvacc > 0:
                risk_ratio_inf = inf_rate_vacc / inf_rate_unvacc
                ve_infection = (1 - risk_ratio_inf) * 100
                
                # For CI, we need counts, not rates. Reconstruct approximate counts.
                # This is an approximation as the rates are per 100k and total cases are summed.
                cases_vacc_inf = data_inf['total_cases'] * (data_inf['vaccinated_pop'] / (data_inf['vaccinated_pop'] + data_inf['unvaccinated_pop']))
                cases_unvacc_inf = data_inf['total_cases'] * (data_inf['unvaccinated_pop'] / (data_inf['vaccinated_pop'] + data_inf['unvaccinated_pop']))
                
                ci_infection = calculate_ci(risk_ratio_inf, data_inf['vaccinated_pop'], data_inf['unvaccinated_pop'], cases_vacc_inf, cases_unvacc_inf)
        
        # VE against death
        if vaccine in death_data:
            data_death = death_data[vaccine]
            death_rate_vacc = data_death['death_rate_vaccinated']
            death_rate_unvacc = data_death['death_rate_unvaccinated']
            
            if death_rate_unvacc > 0:
                risk_ratio_death = death_rate_vacc / death_rate_unvacc
                ve_death = (1 - risk_ratio_death) * 100
                
                # For CI, reconstruct approximate counts
                cases_vacc_death = data_death['total_deaths'] * (data_death['vaccinated_pop'] / (data_death['vaccinated_pop'] + data_death['unvaccinated_pop']))
                cases_unvacc_death = data_death['total_deaths'] * (data_death['unvaccinated_pop'] / (data_death['vaccinated_pop'] + data_death['unvaccinated_pop']))
                
                ci_death = calculate_ci(risk_ratio_death, data_death['vaccinated_pop'], data_death['unvaccinated_pop'], cases_vacc_death, cases_unvacc_death)
        
        ve_results[vaccine] = {
            'VE_infection': ve_infection,
            'VE_death': ve_death,
            'CI_infection': ci_infection,
            'CI_death': ci_death
        }
        
    return ve_results

### Phase 4: Statistical Validation & Output

# Determine vaccination start dates for relevant countries
vaccination_start_date = df_vacc_country.groupby('country')['date'].min().to_dict()

# Identify countries with significant vaccination data to analyze
# For simplicity, let's pick a few countries that appear in manufacturer data
countries_with_vacc_data = df_vacc_manufacturer['country'].unique().tolist()
countries_to_analyze_approach1 = [c for c in countries_with_vacc_data if c in vaccination_start_date and get_population(c) > 0]

print(f"Countries selected for Approach 1 analysis: {countries_to_analyze_approach1}")

# Calculate infection and death rates
infection_rates_by_mfr = calculate_infection_rates_by_manufacturer(countries_to_analyze_approach1, vaccination_start_date)
death_rates_by_mfr = calculate_death_rates_by_manufacturer(countries_to_analyze_approach1, vaccination_start_date)

print("\nInfection Rates by Manufacturer:")
for mfr, data in infection_rates_by_mfr.items():
    print(f"  {mfr} ({data['country']}): Vaccinated Rate: {data['infection_rate_vaccinated']:.2f}/100k, Unvaccinated Rate: {data['infection_rate_unvaccinated']:.2f}/100k")

print("\nDeath Rates by Manufacturer:")
for mfr, data in death_rates_by_mfr.items():
    print(f"  {mfr} ({data['country']}): Vaccinated Rate: {data['death_rate_vaccinated']:.2f}/100k, Unvaccinated Rate: {data['death_rate_unvaccinated']:.2f}/100k")

# Calculate Vaccine Effectiveness
ve_results_approach1 = calculate_vaccine_effectiveness(infection_rates_by_mfr, death_rates_by_mfr)

print("\n--- Approach 1 Results: Vaccine Effectiveness ---")
for vaccine, results in ve_results_approach1.items():
    print(f"\nManufacturer: {vaccine}")
    print(f"  VE against infection: {results['VE_infection']:.2f}% (95% CI: {results['CI_infection'][0]:.2f}% - {results['CI_infection'][1]:.2f}%) ")
    print(f"  VE against death: {results['VE_death']:.2f}% (95% CI: {results['CI_death'][0]:.2f}% - {results['CI_death'][1]:.2f}%) ")

# Relative Performance Ranking
manufacturer_ranking_approach1 = sorted([item for item in ve_results_approach1.items() if not np.isnan(item[1]['VE_infection'])],
                                        key=lambda x: x[1]['VE_infection'],
                                        reverse=True)

print("\n--- Approach 1: Relative Performance Ranking (by VE against infection) ---")
for rank, (vaccine, results) in enumerate(manufacturer_ranking_approach1):
    print(f"{rank + 1}. {vaccine}: {results['VE_infection']:.2f}% effective against infection")

print("\nNote on Approach 1 Assumptions and Limitations:")
print("- Population data is based on a placeholder dictionary. Real-world analysis requires accurate demographic data.")
print("- Case/death attribution to vaccinated/unvaccinated groups assumes proportional distribution based on population segments, which is a strong epidemiological assumption without individual-level data.")
print("- Confidence intervals are Wald intervals, which are approximations and may not be suitable for all scenarios, especially with small counts.")
print("- 'Total number of vaccinations' from manufacturer data is used to infer manufacturer market share, but 'people_fully_vaccinated' from country data is used for the overall vaccinated population denominator to avoid double-counting doses.")
print("- Dominant vaccine is determined based on total vaccinations up to 3 months post-rollout start to allow for initial distribution.")

Countries selected for Approach 1 analysis: ['Argentina', 'Austria', 'Belgium', 'Bulgaria', 'Chile', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Ecuador', 'Estonia', 'Finland', 'France', 'Germany', 'Hong Kong', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Nepal', 'Netherlands', 'Norway', 'Peru', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United States', 'Uruguay']


KeyError: 'Date'

## Approach 2: Time-to-Impact Performance Analysis

In [ ]:
### Phase 1: Vaccination Impact Timeline

def identify_rollout_phases(country, manufacturer=None):
    """
    Identifies key vaccination rollout milestones for a country, optionally by manufacturer.
    """
    if manufacturer:
        vacc_data = df_vacc_manufacturer[
            (df_vacc_manufacturer['country'] == country) &
            (df_vacc_manufacturer['manufacturer'] == manufacturer)
        ].sort_values('date')
        total_vacc_col = 'total_vaccinations'
    else:
        vacc_data = df_vacc_country[
            (df_vacc_country['country'] == country)
        ].sort_values('date')
        total_vacc_col = 'total_vaccinations'
    
    if vacc_data.empty:
        return None, None, None, None

    rollout_start = vacc_data[vacc_data[total_vacc_col] > 0]['date'].min()
    if pd.isna(rollout_start):
        return None, None, None, None

    total_pop = get_population(country)
    
    # Find date when 10% of population is vaccinated (mass vaccination)
    mass_vaccination_date = vacc_data[vacc_data[total_vacc_col] > total_pop * 0.1]['date'].min()
    
    # Find date when 50% of population is vaccinated (high coverage)
    high_coverage_date = vacc_data[vacc_data[total_vacc_col] > total_pop * 0.5]['date'].min()
    
    return rollout_start, mass_vaccination_date, high_coverage_date, total_pop

def calculate_baseline_metrics(country, rollout_start):
    """
    Calculates pre-vaccination baseline metrics (case and death rates).
    """
    if pd.isna(rollout_start):
        return np.nan, np.nan
        
    pre_vacc_period = df_covid_comprehensive[
        (df_covid_comprehensive['country'] == country) &
        (df_covid_comprehensive['Date'] < rollout_start) & # Corrected from 'date' to 'Date'
        (df_covid_comprehensive['Date'] >= rollout_start - timedelta(days=60)) # 60 days baseline # Corrected from 'date' to 'Date'
    ].copy()
    
    if pre_vacc_period.empty:
        return np.nan, np.nan
        
    pre_vacc_period['daily_cases'] = pre_vacc_period['confirmed_cases'].diff().fillna(0).clip(lower=0)
    pre_vacc_period['daily_deaths'] = pre_vacc_period['deaths_cases'].diff().fillna(0).clip(lower=0)
    
    baseline_case_rate = pre_vacc_period['daily_cases'].mean()
    baseline_death_rate = pre_vacc_period['daily_deaths'].mean()
    
    return baseline_case_rate, baseline_death_rate

### Phase 2: Performance Metric Calculation

def calculate_time_to_impact(country, manufacturer, rollout_start, baseline_cases):
    """
    Calculates the time (in days) to achieve a 50% reduction in daily cases.
    """
    if pd.isna(rollout_start) or pd.isna(baseline_cases) or baseline_cases == 0:
        return np.nan
        
    post_vacc_data = df_covid_comprehensive[
        (df_covid_comprehensive['country'] == country) &
        (df_covid_comprehensive['Date'] >= rollout_start) # Corrected from 'date' to 'Date'
    ].copy()
    
    if post_vacc_data.empty:
        return np.nan
        
    post_vacc_data['daily_cases'] = post_vacc_data['confirmed_cases'].diff().fillna(0).clip(lower=0)
    
    # Calculate case reduction percentage, ensuring it doesn't go above 100% or below 0%
    post_vacc_data['case_reduction_pct'] = (
        (baseline_cases - post_vacc_data['daily_cases']) / baseline_cases * 100
    ).clip(lower=0, upper=100)
    
    # Find the first date where case reduction is >= 50%
    time_to_50_reduction_df = post_vacc_data[
        post_vacc_data['case_reduction_pct'] >= 50
    ]
    
    if time_to_50_reduction_df.empty:
        return np.nan # 50% reduction not achieved
        
    time_to_50_reduction = time_to_50_reduction_df['Date'].min() # Corrected from 'date' to 'Date'
    
    days_to_impact = (time_to_50_reduction - rollout_start).days
    
    return days_to_impact

def calculate_sustained_effectiveness(country, manufacturer, rollout_start, baseline_cases):
    """
    Calculates sustained effectiveness over time periods.
    """
    effectiveness_timeline = {}
    if pd.isna(rollout_start) or pd.isna(baseline_cases) or baseline_cases == 0:
        return {f'{months}_months': np.nan for months in [1, 3, 6, 9, 12]}

    for months in [1, 3, 6, 9, 12]:
        period_start = rollout_start + timedelta(days=30*months)
        period_end = period_start + timedelta(days=30) # Evaluate over a 30-day window
        
        period_data = df_covid_comprehensive[
            (df_covid_comprehensive['country'] == country) &
            (df_covid_comprehensive['Date'].between(period_start, period_end)) # Corrected from 'date' to 'Date'
        ].copy()
        
        if period_data.empty:
            effectiveness_timeline[f'{months}_months'] = np.nan
            continue
            
        period_data['daily_cases'] = period_data['confirmed_cases'].diff().fillna(0).clip(lower=0)
        avg_case_rate = period_data['daily_cases'].mean()
        
        effectiveness = (baseline_cases - avg_case_rate) / baseline_cases * 100
        effectiveness_timeline[f'{months}_months'] = max(0, effectiveness) # Cap at 0% if cases increased
        
    return effectiveness_timeline

### Phase 3: Comparative Performance Scoring

def create_performance_index(manufacturer_results):
    """
    Creates a combined performance index based on speed, durability, and peak effectiveness.
    """
    performance_scores = {}
    
    for manufacturer, results in manufacturer_results.items():
        speed_score = np.nan
        durability_score = np.nan
        peak_effectiveness = np.nan
        overall_score = np.nan
        
        # Speed score (inverse of days to impact)
        if not pd.isna(results['days_to_impact']) and results['days_to_impact'] > 0:
            speed_score = 100 / results['days_to_impact']
        elif not pd.isna(results['days_to_impact']) and results['days_to_impact'] == 0:
            speed_score = 100 # Immediate impact
        
        # Durability score (slope of effectiveness decline)
        effectiveness_values = [v for v in results['effectiveness_timeline'].values() if not pd.isna(v)]
        if len(effectiveness_values) >= 2:
            # Fit a linear regression to effectiveness over time
            x = np.array(range(len(effectiveness_values)))
            y = np.array(effectiveness_values)
            
            # Handle cases where all y values are the same (slope is 0)
            if np.std(y) == 0:
                slope = 0
            else:
                slope = np.polyfit(x, y, 1)[0]
            
            # Durability: higher score for less decline (or even increase)
            # Scale slope to a reasonable range, e.g., -10 to 10, then map to 0-100
            # A slope of 0 means perfect durability (100 score)
            # A slope of -10 (steep decline) would be 0 score
            # A slope of 10 (steep increase) would be 100 score (capped)
            durability_score = 100 - (abs(slope) * 5) # Adjust multiplier as needed
            if slope > 0: # Reward increasing effectiveness
                durability_score = 100 + (slope * 5)
            durability_score = np.clip(durability_score, 0, 100) # Cap between 0 and 100
        elif len(effectiveness_values) == 1:
            durability_score = 100 # If only one point, assume perfect durability for that point
        
        # Peak effectiveness score
        if effectiveness_values:
            peak_effectiveness = max(effectiveness_values)
        
        # Combined performance index (only if all components are available)
        if not pd.isna(speed_score) and not pd.isna(durability_score) and not pd.isna(peak_effectiveness):
            overall_score = (speed_score * 0.3 + durability_score * 0.4 + peak_effectiveness * 0.3)
        
        performance_scores[manufacturer] = {
            'speed_score': speed_score,
            'durability_score': durability_score,
            'peak_effectiveness': peak_effectiveness,
            'overall_performance_index': overall_score
        }
        
    return performance_scores

### Phase 4: Output

# Identify countries with significant vaccination data to analyze for Approach 2
# We need countries where we can identify a dominant vaccine and have comprehensive data
countries_to_analyze_approach2 = [c for c in countries_with_vacc_data if c in vaccination_start_date and c in df_covid_comprehensive['country'].unique()]

manufacturer_performance_results = {}

for country in countries_to_analyze_approach2:
    dominant_vaccine = get_dominant_vaccine(country) # Use overall dominant vaccine for time-to-impact
    if not dominant_vaccine:
        print(f"Skipping {country} for Approach 2: No dominant vaccine found.")
        continue
        
    rollout_start, _, _, _ = identify_rollout_phases(country, dominant_vaccine)
    if pd.isna(rollout_start):
        print(f"Skipping {country} for Approach 2: Rollout start not found.")
        continue
        
    baseline_cases, _ = calculate_baseline_metrics(country, rollout_start)
    if pd.isna(baseline_cases) or baseline_cases == 0:
        print(f"Skipping {country} for Approach 2: Baseline cases not found or zero.")
        continue
        
    days_to_impact = calculate_time_to_impact(country, dominant_vaccine, rollout_start, baseline_cases)
    effectiveness_timeline = calculate_sustained_effectiveness(country, dominant_vaccine, rollout_start, baseline_cases)
    
    if dominant_vaccine not in manufacturer_performance_results:
        manufacturer_performance_results[dominant_vaccine] = {
            'days_to_impact': [],
            'effectiveness_timeline': {m: [] for m in [1, 3, 6, 9, 12]}
        }
    
    if not pd.isna(days_to_impact):
        manufacturer_performance_results[dominant_vaccine]['days_to_impact'].append(days_to_impact)
    
    for month, eff in effectiveness_timeline.items():
        if not pd.isna(eff):
            manufacturer_performance_results[dominant_vaccine]['effectiveness_timeline'][month].append(eff)

# Average results across countries for each manufacturer
averaged_manufacturer_results = {}
for mfr, data in manufacturer_performance_results.items():
    avg_days_to_impact = np.mean(data['days_to_impact']) if data['days_to_impact'] else np.nan
    avg_effectiveness_timeline = {month: np.mean(eff_list) if eff_list else np.nan
                                  for month, eff_list in data['effectiveness_timeline'].items()}
    averaged_manufacturer_results[mfr] = {
        'days_to_impact': avg_days_to_impact,
        'effectiveness_timeline': avg_effectiveness_timeline
    }

performance_scores_approach2 = create_performance_index(averaged_manufacturer_results)

print("\n--- Approach 2 Results: Time-to-Impact Performance Analysis ---")
for vaccine, scores in performance_scores_approach2.items():
    print(f"\nManufacturer: {vaccine}")
    print(f"  Speed Score: {scores['speed_score']:.2f}")
    print(f"  Durability: {scores['durability_score']:.2f}")
    print(f"  Peak: {scores['peak_effectiveness']:.2f}")
    print(f"  Overall Index: {scores['overall_performance_index']:.2f}")

print("\nNote on Approach 2 Assumptions and Limitations:")
print("- Population data is based on a placeholder dictionary.")
print("- 'Time to 50% case reduction' is calculated based on overall country cases, not specific to vaccinated individuals.")
print("- Durability score uses a linear fit, which may not accurately represent complex effectiveness decay patterns.")
print("- Averaging results across countries for a manufacturer assumes similar conditions, which may not hold true.")

## Approach 3: Breakthrough Rate Performance Calculation

In [ ]:
### Phase 1: Vaccination Status Estimation

def estimate_testing_population_vacc_status(country, date, manufacturer):
    """
    Estimates the number of vaccinated and unvaccinated individuals within the daily tested population.
    Assumes testing rates are proportional to vaccination status in the general population.
    """
    total_population = get_population(country)
    if total_population == 0:
        return 0, 0, 0 # Return zeros if population is zero

    # Get vaccination coverage by manufacturer at specific date
    vacc_coverage_mfr = df_vacc_manufacturer[
        (df_vacc_manufacturer['country'] == country) &
        (df_vacc_manufacturer['date'] <= date)
    ].sort_values('date', ascending=False).head(1)
    
    total_vaccinations_mfr = vacc_coverage_mfr['total_vaccinations'].iloc[0] if not vacc_coverage_mfr.empty else 0
    
    # Get overall fully vaccinated population for the country
    vacc_coverage_country = df_vacc_country[
        (df_vacc_country['country'] == country) &
        (df_vacc_country['date'] <= date)
    ].sort_values('date', ascending=False).head(1)
    
    people_fully_vaccinated = vacc_coverage_country['people_fully_vaccinated'].iloc[0] if not vacc_coverage_country.empty else 0
    
    # Cap vaccinated population at total_population
    people_fully_vaccinated = min(people_fully_vaccinated, total_population)

    unvaccinated_pop_overall = max(0, total_population - people_fully_vaccinated)
    
    # Estimate the proportion of the dominant manufacturer's vaccine among the fully vaccinated
    # This is a simplification: assumes total_vaccinations by manufacturer is proportional to fully vaccinated individuals
    if people_fully_vaccinated > 0 and total_vaccinations_mfr > 0:
        # This is a very rough estimate. Ideally, we'd need data on fully vaccinated by manufacturer.
        # For now, we'll use the manufacturer's share of total vaccinations as a proxy for their share of fully vaccinated.
        total_vacc_all_mfr = df_vacc_manufacturer[
            (df_vacc_manufacturer['country'] == country) &
            (df_vacc_manufacturer['date'] <= date)
        ].groupby('manufacturer')['total_vaccinations'].max().sum()
        
        if total_vacc_all_mfr > 0:
            mfr_proportion_of_total_vacc = total_vaccinations_mfr / total_vacc_all_mfr
            vaccinated_pop_mfr = people_fully_vaccinated * mfr_proportion_of_total_vacc
        else:
            vaccinated_pop_mfr = 0
    else:
        vaccinated_pop_mfr = 0

    # Get daily tests for the country and date
    daily_tests_data = df_testing[
        (df_testing['country'] == country) &
        (df_testing['date'] == date)
    ].sort_values('date', ascending=False).head(1)
    
    daily_tests = daily_tests_data['daily_change'].iloc[0] if not daily_tests_data.empty else 0
    
    # Distribute daily tests proportionally based on overall vaccination status
    if total_population > 0:
        prop_vaccinated = people_fully_vaccinated / total_population
        prop_unvaccinated = unvaccinated_pop_overall / total_population
    else:
        prop_vaccinated = 0
        prop_unvaccinated = 0
        
    estimated_vaccinated_tested = daily_tests * prop_vaccinated
    estimated_unvaccinated_tested = daily_tests * prop_unvaccinated
    
    return estimated_vaccinated_tested, estimated_unvaccinated_tested, vaccinated_pop_mfr

### Phase 2: Breakthrough Rate Calculation

# Epidemiological assumption: Factor to estimate breakthrough infections from total positive tests
# This is a simplification. Real breakthrough data is complex and often requires specific surveillance.
BREAKTHROUGH_FACTOR = 0.7 # Assume 70% of vaccinated positive tests are breakthroughs

def calculate_breakthrough_rates(countries_to_analyze, dominant_vaccine_map):
    """
    Calculates manufacturer-specific breakthrough rates and unvaccinated rates.
    """
    breakthrough_results = {}
    
    for country in countries_to_analyze:
        dominant_vaccine = dominant_vaccine_map.get(country)
        if not dominant_vaccine:
            print(f"Skipping {country} for breakthrough rates: No dominant vaccine found.")
            continue
            
        testing_data = df_testing[df_testing['country'] == country].sort_values('date').copy()
        
        if testing_data.empty:
            print(f"Skipping {country} for breakthrough rates: No testing data.")
            continue
            
        country_breakthrough_rates = []
        
        for index, row in testing_data.iterrows():
            date = row['date']
            # Ensure positive_rate is not NaN and daily_change is not NaN
            if pd.isna(row['positive_rate']) or pd.isna(row['daily_change']):
                continue
            
            positive_tests = row['daily_change'] * (row['positive_rate'] / 100)
            
            estimated_vaccinated_tested, estimated_unvaccinated_tested, vaccinated_pop_mfr = \
                estimate_testing_population_vacc_status(country, date, dominant_vaccine)
            
            if (estimated_vaccinated_tested + estimated_unvaccinated_tested) == 0:
                continue
            
            # Estimate positive tests in vaccinated vs unvaccinated groups
            # This is a critical assumption: assumes positive rate applies equally to tested groups
            prop_vacc_tested = estimated_vaccinated_tested / (estimated_vaccinated_tested + estimated_unvaccinated_tested)
            prop_unvacc_tested = estimated_unvaccinated_tested / (estimated_vaccinated_tested + estimated_unvaccinated_tested)
            
            estimated_positive_vacc = positive_tests * prop_vacc_tested
            estimated_positive_unvacc = positive_tests * prop_unvacc_tested
            
            # Estimate breakthrough infections from vaccinated positive tests
            estimated_breakthrough = estimated_positive_vacc * BREAKTHROUGH_FACTOR
            
            breakthrough_rate = (estimated_breakthrough / vaccinated_pop_mfr) * 100000 if vaccinated_pop_mfr > 0 else 0
            unvacc_rate = (estimated_unvacc_positive / estimated_unvaccinated_tested) * 100000 if estimated_unvaccinated_tested > 0 else 0
            
            country_breakthrough_rates.append({
                'date': date,
                'breakthrough_rate': breakthrough_rate,
                'unvaccinated_rate': unvacc_rate
            })
        
        if country_breakthrough_rates:
            if dominant_vaccine not in breakthrough_results:
                breakthrough_results[dominant_vaccine] = []
            breakthrough_results[dominant_vaccine].extend(country_breakthrough_rates)
            
    return breakthrough_results

def calculate_breakthrough_prevention_effectiveness(breakthrough_data):
    """
    Calculates Breakthrough Prevention Effectiveness (BPE).
    """
    bpe_results = {}
    
    for vaccine, data_list in breakthrough_data.items():
        if not data_list:
            bpe_results[vaccine] = {
                'breakthrough_rate_per_100k': np.nan,
                'unvaccinated_rate_per_100k': np.nan,
                'breakthrough_prevention_effectiveness': np.nan
            }
            continue
            
        df = pd.DataFrame(data_list)
        
        avg_breakthrough_rate = df['breakthrough_rate'].mean()
        avg_unvaccinated_rate = df['unvaccinated_rate'].mean()
        
        bpe = np.nan
        if avg_unvaccinated_rate > 0:
            bpe = (1 - avg_breakthrough_rate / avg_unvaccinated_rate) * 100
        else:
            bpe = 0 # If no unvaccinated cases, assume 100% effectiveness if breakthrough rate is also 0
            if avg_breakthrough_rate > 0: # If breakthrough exists but no unvacc, something is wrong or BPE is negative
                bpe = -100 # Indicate issue or very low effectiveness
        
        bpe_results[vaccine] = {
            'breakthrough_rate_per_100k': avg_breakthrough_rate,
            'unvaccinated_rate_per_100k': avg_unvaccinated_rate,
            'breakthrough_prevention_effectiveness': bpe
        }
        
    return bpe_results

### Phase 3: Testing Pattern Performance Analysis

def analyze_test_positivity_performance(countries_to_analyze, vaccination_start_date_map):
    """
    Analyzes changes in test positivity rates pre and post-vaccination.
    """
    positivity_performance = {}
    
    for country in countries_to_analyze:
        dominant_vaccine = get_dominant_vaccine(country)
        if not dominant_vaccine:
            continue
            
        vacc_start = vaccination_start_date_map.get(country)
        if pd.isna(vacc_start):
            continue
            
        testing_data_country = df_testing[df_testing['country'] == country].copy()
        
        if testing_data_country.empty:
            continue
            
        # Pre-vaccination period: 60 days before rollout start
        pre_vacc_period_data = testing_data_country[
            (testing_data_country['date'] < vacc_start) &
            (testing_data_country['date'] >= vacc_start - timedelta(days=60))
        ]
        pre_vacc_positivity = pre_vacc_period_data['positive_rate'].mean()
        
        # Post-vaccination period: 42 days after rollout start (allowing for immunity development) for 60 days
        post_vacc_period_start = vacc_start + timedelta(days=42)
        post_vacc_period_end = post_vacc_period_start + timedelta(days=60)
        
        post_vacc_period_data = testing_data_country[
            (testing_data_country['date'] >= post_vacc_period_start) &
            (testing_data_country['date'] <= post_vacc_period_end)
        ]
        post_vacc_positivity = post_vacc_period_data['positive_rate'].mean()
        
        positivity_reduction = np.nan
        if not pd.isna(pre_vacc_positivity) and pre_vacc_positivity > 0:
            positivity_reduction = (pre_vacc_positivity - post_vacc_positivity) / pre_vacc_positivity * 100
            positivity_reduction = np.clip(positivity_reduction, 0, 100) # Cap at 100%
        
        positivity_performance[dominant_vaccine] = {
            'country': country,
            'pre_vaccination_positivity': pre_vacc_positivity,
            'post_vaccination_positivity': post_vacc_positivity,
            'positivity_reduction_percentage': positivity_reduction
        }
        
    return positivity_performance

### Phase 4: Integrated Performance Scoring

def create_breakthrough_performance_index(bpe_results, positivity_results):
    """
    Creates a comprehensive breakthrough performance index.
    """
    final_scores = {}
    
    all_manufacturers = set(bpe_results.keys()).union(set(positivity_results.keys()))
    
    for vaccine in all_manufacturers:
        bpe_score = np.nan
        positivity_score = np.nan
        breakthrough_performance_index = np.nan
        breakthrough_rate_per_100k = np.nan
        
        if vaccine in bpe_results:
            bpe_score = max(0, bpe_results[vaccine]['breakthrough_prevention_effectiveness'])
            breakthrough_rate_per_100k = bpe_results[vaccine]['breakthrough_rate_per_100k']
            
        if vaccine in positivity_results:
            # Average positivity reduction if multiple countries for same vaccine
            country_pos_reductions = [res['positivity_reduction_percentage'] for res_vacc, res in positivity_results.items() if res_vacc == vaccine and not pd.isna(res['positivity_reduction_percentage'])]
            if country_pos_reductions:
                positivity_score = max(0, np.mean(country_pos_reductions))
            
        if not pd.isna(bpe_score) and not pd.isna(positivity_score):
            breakthrough_performance_index = (bpe_score * 0.7 + positivity_score * 0.3)
        
        final_scores[vaccine] = {
            'breakthrough_prevention_effectiveness': bpe_score,
            'test_positivity_improvement': positivity_score,
            'breakthrough_performance_index': breakthrough_performance_index,
            'breakthrough_rate_per_100k': breakthrough_rate_per_100k
        }
        
    return final_scores

### Output for Approach 3

# Identify countries for Approach 3 analysis (same as Approach 1/2 for consistency)
countries_to_analyze_approach3 = [c for c in countries_with_vacc_data if c in vaccination_start_date and c in df_testing['country'].unique()]

# Map dominant vaccine to country for easier lookup
dominant_vaccine_by_country = {country: get_dominant_vaccine(country) for country in countries_to_analyze_approach3}
dominant_vaccine_by_country = {k: v for k, v in dominant_vaccine_by_country.items() if v is not None}

print(f"Countries selected for Approach 3 analysis: {list(dominant_vaccine_by_country.keys())}")

breakthrough_rates_data = calculate_breakthrough_rates(list(dominant_vaccine_by_country.keys()), dominant_vaccine_by_country)
bpe_results_approach3 = calculate_breakthrough_prevention_effectiveness(breakthrough_rates_data)
positivity_results_approach3 = analyze_test_positivity_performance(list(dominant_vaccine_by_country.keys()), vaccination_start_date)

final_scores_approach3 = create_breakthrough_performance_index(bpe_results_approach3, positivity_results_approach3)

print("\n--- Approach 3 Results: Breakthrough Rate Performance Calculation ---")
for vaccine, scores in final_scores_approach3.items():
    print(f"\nManufacturer: {vaccine}")
    print(f"  BPE: {scores['breakthrough_prevention_effectiveness']:.2f}%")
    print(f"  Positivity Reduction: {scores['test_positivity_improvement']:.2f}%")
    print(f"  Breakthrough Index: {scores['breakthrough_performance_index']:.2f}")
    print(f"  Breakthrough Rate per 100k: {scores['breakthrough_rate_per_100k']:.2f}")

print("\nNote on Approach 3 Assumptions and Limitations:")
print("- Population data is based on a placeholder dictionary.")
print("- Estimation of vaccinated/unvaccinated individuals within the tested population is a simplification based on overall vaccination rates.")
print("- The `BREAKTHROUGH_FACTOR` is an arbitrary assumption for estimating breakthrough infections from positive tests. Real-world data would require specific surveillance and reporting of breakthrough cases.")
print("- Test positivity analysis assumes changes are primarily due to vaccine impact, not other factors like changes in testing strategy or viral prevalence.")